In [46]:
folder_path = "../data/"

In [47]:
import os

all_pdfs = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]

all_pdfs

['../data/data science learning - LLM training and inference.pdf',
 '../data/DeepSeek-R1.pdf']

In [48]:
import pymupdf 
import re
from bs4 import BeautifulSoup
import unicodedata
from collections import Counter


def clean_text(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    
    # Only use BeautifulSoup if text looks like HTML
    if bool(re.search(r"<[^>]+>", text)):
        text = BeautifulSoup(text, "html.parser").get_text()
    
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Optional: remove non-ASCII
    return text


def extract_text(pdf_path, remove_headers_footers=True):
    doc = pymupdf.open(pdf_path)
    pages_text = []
    header_footer_candidates = []

    for page in doc:
        page_text = page.get_text("text") or ""
        lines = [clean_text(line.strip()) for line in page_text.split("\n") if len(line.strip()) > 10]
        if remove_headers_footers:
            header_footer_candidates.extend(lines[:2] + lines[-2:])
        pages_text.append((page.number + 1, lines))

    repetitive = set()
    if remove_headers_footers:
        counts = Counter(header_footer_candidates)
        repetitive = {line for line, c in counts.items() if c > 0.6 * len(pages_text)}

    text_docs = []
    for page_num, lines in pages_text:
        clean_lines = [ln for ln in lines if ln not in repetitive]
        text_docs.append({
            "metadata": {"page_number": page_num, "source": pdf_path},
            "text": " ".join(clean_lines)
        })

    doc.close()
    return text_docs


def pdfs_loader(paths):
    all_docs = []
    for pdf_path in paths:
        docs = extract_text(pdf_path)
        all_docs.extend(docs)
    return all_docs


In [49]:
docs = pdfs_loader(all_pdfs)

In [50]:
docs

[{'metadata': {'page_number': 1,
   'source': '../data/data science learning - LLM training and inference.pdf'},
  'text': 'data science learning - LLM training and inference i want to learn ,how to large language model work , how to train or fine-tune and how to make efficient model inference , internel architecture and any other components Great   let s build a structured roadmap so you can deeply understand how large language models (LLMs) work, how they are trained/fine-tuned, and how inference is optimized. I ll break this into digestible parts, from foundations to advanced topics. 1. Foundations of LLMs Before diving into training and optimization, you need to be comfortable with the core building blocks. Neural Networks Basics Perceptron   Multi-Layer Perceptron (MLP) Backpropagation & Gradient Descent Sequence Models Evolution RNNs, LSTMs, GRUs   limitations (vanishing gradients, poor long- range memory) Self-Attention introduced in  Attention is All You Need  (2017) Transforme

In [68]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter
import hashlib

def get_text_splitter(strategy="recursive", chunk_size=800, chunk_overlap=150, model_name="gpt-4o"):
    if strategy == "token":
        return TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, model_name=model_name)
    # default: character recursive (works well)
    return RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

splitter = get_text_splitter(strategy="recursive", chunk_size=800, chunk_overlap=200)

def generate_chunk_id(text: str) -> str:
    """
    Generate a stable unique ID for a text chunk using file name, page, and text hash.
    """
    # short hash of text to ensure uniqueness even if chunk sizes vary
    text_hash = hashlib.md5(text.encode('utf-8')).hexdigest()
    return text_hash


def chunking_doc(documents, splitter=splitter):
    chunking_dataset = []
    for document in documents:
        texts = splitter.split_text(document["text"])
        for text in texts:
            chunking_dataset.append({
                "text": text,
                "metadata": document["metadata"],
                "id": generate_chunk_id(
                    text=text
                )
            })
    return chunking_dataset

In [69]:
chunk_doc = chunking_doc(docs)

In [70]:
len(docs), len(chunk_doc)

(41, 123)

In [71]:
chunk_doc

[{'text': 'data science learning - LLM training and inference i want to learn ,how to large language model work , how to train or fine-tune and how to make efficient model inference , internel architecture and any other components Great   let s build a structured roadmap so you can deeply understand how large language models (LLMs) work, how they are trained/fine-tuned, and how inference is optimized. I ll break this into digestible parts, from foundations to advanced topics. 1. Foundations of LLMs Before diving into training and optimization, you need to be comfortable with the core building blocks. Neural Networks Basics Perceptron   Multi-Layer Perceptron (MLP) Backpropagation & Gradient Descent Sequence Models Evolution RNNs, LSTMs, GRUs   limitations (vanishing gradients, poor long- range',
  'metadata': {'page_number': 1,
   'source': '../data/data science learning - LLM training and inference.pdf'},
  'id': 'cc3d884f6866ceb2b61c28e45ba9ebe8'},
 {'text': 'Neural Networks Basics P

In [72]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "models/gemini-2.5-flash-lite")


In [75]:
import chromadb

client = chromadb.PersistentClient(path="./store_emb")

collection = client.get_or_create_collection(
    name="rag_collection", 
    configuration={
        "hnsw": {
            "space": "cosine",
            "ef_construction": 200,
            "max_neighbors":32
        }
    }
)

In [76]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
import time
from tqdm import tqdm

os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

def gemini_embedding_batch(texts, batch_size=2):
    result_emb = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        result_emb.extend(embeddings.embed_documents(batch))
        time.sleep(2)
    return result_emb

In [77]:
def embed_and_store(docs):
    # chunking 
    texts = [doc['text'] for doc in docs]
    metadatas = [doc['metadata'] for doc in docs]
    embs = gemini_embedding_batch(texts)

    
    ids = [doc['id'] for doc in docs]
    
    collection.add(
        documents=texts,
        metadatas=metadatas,
        embeddings=embs,
        ids=ids
    )

In [78]:
embed_and_store(chunk_doc)

100%|██████████| 62/62 [02:49<00:00,  2.73s/it]
